In [1]:
import inspect

# Basics

In [15]:
from langchain.prompts import (
    ChatPromptTemplate, 
    HumanMessagePromptTemplate, 
    AIMessagePromptTemplate, 
    SystemMessagePromptTemplate
)

`ChatPromptTemplate` is the base class for all chat prompts

It is constructed with a list of message prompt templates (`HumanMessagePromptTemplate`, `AIMessagePromptTemplate`, `SystemMessagePromptTemplate`)

In [23]:
# Most convenient way to setup a chat prompt template: 
# 1. use `from_template` for creating message prompt templates
# 2. use `from_messages` for creating final chat prompt template

# Create system message prompt template
system_message_template = "Your are a translator. Given a user input in {source_language}, translate it into {target_language}."
system_message_prompt_template = SystemMessagePromptTemplate.from_template(system_message_template)

# Create human message prompt template
human_message_template = "{input}"
human_message_prompt_template = HumanMessagePromptTemplate.from_template(human_message_template)

# Create chat prompt template
chat_prompt_template = ChatPromptTemplate.from_messages([system_message_prompt_template, human_message_prompt_template])
chat_prompt_template

ChatPromptTemplate(input_variables=['target_language', 'input', 'source_language'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['source_language', 'target_language'], output_parser=None, partial_variables={}, template='Your are a translator. Given a user input in {source_language}, translate it into {target_language}.', template_format='f-string', validate_template=True), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], output_parser=None, partial_variables={}, template='{input}', template_format='f-string', validate_template=True), additional_kwargs={})])

In [26]:
# Create prompt from prompt template
chat_prompt = chat_prompt_template.format(source_language="English", target_language="French", input="Hello, how are you?")
chat_prompt

'System: Your are a translator. Given a user input in English, translate it into French.\nHuman: Hello, how are you?'

In [31]:
# `format` directly converts the prompt template into a string, therefore for chat prompt templates,
# it may be more practical to first use the  `format_prompt` method, and then create the output format
# we want based on the resulting `ChatPromptValue` object 
chat_prompt = chat_prompt_template.format_prompt(source_language="English", target_language="French", input="Hello, how are you?")
chat_prompt

ChatPromptValue(messages=[SystemMessage(content='Your are a translator. Given a user input in English, translate it into French.', additional_kwargs={}), HumanMessage(content='Hello, how are you?', additional_kwargs={}, example=False)])

In [32]:
# i.e. a list of messages using the `to_messages` method
chat_prompt.to_messages()

[SystemMessage(content='Your are a translator. Given a user input in English, translate it into French.', additional_kwargs={}),
 HumanMessage(content='Hello, how are you?', additional_kwargs={}, example=False)]

In [30]:
# i.e. a list of messages using the `to_string` method --> similar to if we had used `format` directly
chat_prompt.to_string()

'System: Your are a translator. Given a user input in English, translate it into French.\nHuman: Hello, how are you?'

In [36]:
# If we want to create a message prompt template without using the `from_template` method, we need to
# create a PromptTemplate object first, and then pass it to the constructor of the message prompt template
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate(
    template="Your are a translator. Given a user input in {source_language}, translate it into {target_language}.",
    input_variables=["source_language", "target_language"]
)
system_message_prompt_template = SystemMessagePromptTemplate(prompt=prompt_template)
system_message_prompt_template

SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['source_language', 'target_language'], output_parser=None, partial_variables={}, template='Your are a translator. Given a user input in {source_language}, translate it into {target_language}.', template_format='f-string', validate_template=True), additional_kwargs={})

In [37]:
# If we want to create a chat prompt template without using the `from_messages` method, we need to
# pass a list of message prompt templates and a list of input variables to the constructor of the chat 
# prompt template
chat_prompt_template = ChatPromptTemplate(
    messages=[system_message_prompt_template, human_message_prompt_template],
    input_variables=["source_language", "target_language", "input"]
)
chat_prompt_template

ChatPromptTemplate(input_variables=['source_language', 'target_language', 'input'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['source_language', 'target_language'], output_parser=None, partial_variables={}, template='Your are a translator. Given a user input in {source_language}, translate it into {target_language}.', template_format='f-string', validate_template=True), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], output_parser=None, partial_variables={}, template='{input}', template_format='f-string', validate_template=True), additional_kwargs={})])

In [46]:
# Some of the methods that are available for standard LLM models are not yet available 
# for chat models, i.e. partial formatting
try:
    chat_prompt_template.partial(source_language="English", target_language="French")
except NotImplementedError as e:
    print("😢")

😢


# Message Placeholders

In [53]:
from langchain.prompts import MessagesPlaceholder
from langchain.schema import HumanMessage, AIMessage

Message placeholders are a very powerful idea that makes prompting for chat models much more flexible

In [54]:
human_message_template = "Summarize the previous conversation in {num_sentences} sentences."
human_message_prompt_template = HumanMessagePromptTemplate.from_template(human_message_template)
chat_prompt_template = ChatPromptTemplate.from_messages(
    messages=[MessagesPlaceholder(variable_name="conversation"), human_message_prompt_template]
)
chat_prompt_template

ChatPromptTemplate(input_variables=['conversation', 'num_sentences'], output_parser=None, partial_variables={}, messages=[MessagesPlaceholder(variable_name='conversation'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['num_sentences'], output_parser=None, partial_variables={}, template='Summarize the previous conversation in {num_sentences} sentences.', template_format='f-string', validate_template=True), additional_kwargs={})])

In [58]:
chat_prompt_value = chat_prompt_template.format_prompt(
    conversation=[
        AIMessage(content="Hi, I am your personal assistant. How can I help you?"),
        HumanMessage(content="I would like to book a flight to Paris."),
        AIMessage(content="Sure, when would you like to travel?"),
        HumanMessage(content="Next week."),
        AIMessage(content="Great, I found a flight for you. Do you want to book it?"),
        HumanMessage(content="Yes, please."),
        AIMessage(content="Done. Your flight is booked.")
    ],
    num_sentences=2
)
print(chat_prompt_value.to_string())

AI: Hi, I am your personal assistant. How can I help you?
Human: I would like to book a flight to Paris.
AI: Sure, when would you like to travel?
Human: Next week.
AI: Great, I found a flight for you. Do you want to book it?
Human: Yes, please.
AI: Done. Your flight is booked.
Human: Summarize the previous conversation in 2 sentences.
